In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 11.2 MB/s eta 0:00:00


In [30]:
import gensim
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from spacy.lang.es import Spanish
nlp = Spanish()
from datasets import load_dataset
import re
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr

In [3]:
train = load_dataset("stsb_multi_mt", name="es", split="dev")

In [4]:
print(type(train))
print(type(train[0]))
print(type(train['sentence1']))
print(type(train['similarity_score']))
print(train)
print(train[0])
print(train[1238])
print(train[0]["sentence1"])

<class 'datasets.arrow_dataset.Dataset'>
<class 'dict'>
<class 'list'>
<class 'list'>
Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 1500
})
{'sentence1': 'Un hombre con un casco está bailando.', 'sentence2': 'Un hombre con un casco está bailando.', 'similarity_score': 5.0}
{'sentence1': 'Los equipos trabajaron para instalar una nueva alcantarilla y preparar la autopista para que los automovilistas pudieran usar los carriles hacia el este para viajar, ya que las nubes de tormenta amenazaban con arrojar más lluvia.', 'sentence2': 'Los equipos trabajaron para instalar una nueva alcantarilla y repavimentar la autopista para que los automovilistas pudieran usar los carriles hacia el este para viajar.', 'similarity_score': 3.3329999446868896}
Un hombre con un casco está bailando.


In [9]:
sentence1 = []
sentence2 = []
sim_scores = []

for length in range (len(train)):
  sentence1.append(train[length]["sentence1"].lower())
  sentence2.append(train[length]["sentence2"].lower())
  sim_scores.append(train[length]["similarity_score"])

for sentence in sentence1:
      # Puntuation 
    sentence = re.sub(r"[^\w\s']+" ,' ', sentence)
    # sentence = re.sub(r"(?<=[^\d\s])\.(?=\s|$)", "", sentence)
    # sentence = re.sub(r"(?<=\w)\.(?=\w)", "", sentence)
    # sentence = re.sub(re".","", sentence)

    #Removing extra spaces
    sentence = re.sub(r"\s+", " ", sentence)

for sentence in sentence2:
      sentence = re.sub(r"[^\w\s']+" ,' ', sentence)
      sentence = re.sub(r"\s+", " ", sentence)


In [10]:
tokenized_sentences1 = []
for sentence in sentence1:
    doc = nlp(sentence)
    tokenized_sentences1.append([token.text for token in doc])

# print(tokenized_sentences1)

In [11]:
tokenized_sentences2 = []
for sentence in sentence2:
    doc = nlp(sentence)
    tokenized_sentences2.append([token.text for token in doc])

# print(tokenized_sentences2)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd /content/drive/My Drive/NLP/PreTrained_models

/content/drive/My Drive/NLP/PreTrained_models


In [ ]:
# !wget http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

In [ ]:
# !gunzip SBW-vectors-300-min5.bin.gz

In [15]:
model = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin', binary=True)
# SBW-vectors-300-min5

In [21]:
sentence_embeddings1 = []

for sentence in tokenized_sentences1:
    sentence_embedding = []
    for word in sentence:
        if word in model:
            sentence_embedding.append(model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(model.vector_size)
    sentence_embeddings1.append(sentence_embedding)
# print(sentence_embeddings[0])
print((sentence_embeddings1[0].shape))

(300,)


In [22]:
sentence_embeddings2 = []

for sentence in tokenized_sentences2:
    sentence_embedding = []
    for word in sentence:
        if word in model:
            sentence_embedding.append(model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(model.vector_size)
    sentence_embeddings2.append(sentence_embedding)
# print(sentence_embeddings[0])
print((sentence_embeddings2[0].shape))

(300,)


In [23]:
from sklearn.metrics.pairwise import cosine_similarity
# sentence_embeddings1 and sentence_embeddings2 are the two lists of sentence embeddings
# of the same length
assert len(sentence_embeddings1) == len(sentence_embeddings2)

# Compute the cosine similarity between corresponding indexed sentences
cos_sim = []
for emb1, emb2 in zip(sentence_embeddings1, sentence_embeddings2):
    similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    cos_sim.append(similarity)


In [31]:
# Normalize sim_scores using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
sim_scores_norm = scaler.fit_transform(np.array(sim_scores).reshape(-1, 1)).flatten()


# cos_sim and similarity_score are the two lists of similarity scores
# of the same length
assert len(cos_sim) == len(sim_scores)

# Calculate the Pearson correlation coefficient and its p-value
corr, p_value = pearsonr(cos_sim, sim_scores_norm)

print(f"Pearson correlation coefficient: {corr:.3f}")
print(f"P-value: {p_value:.3f}")


Pearson correlation coefficient: 0.633
P-value: 0.000
